# Predicting a players likelihood of being drafted based on Juniors Statistics

Using Decision trees and random forest classifiers we are attempting to predict whether or not a given player will be drafted to by an NHL teams.
### Pre-analysis Prediction
I expect the results to be less accurate at predicting a drafted players likeliehood of making it to the NHL. Due to the fact the some players will play long careers in the Juniors only to never get drafted while others might only play half a season and get drafted and moved into professional leagues faster.


In [1]:
# Dependencies
from sklearn import tree
import pandas as pd
import os 
from sklearn.model_selection import train_test_split

In [22]:
# Load in data
df = pd.read_csv('data_sets/juniors_data.csv')
df.head()

LEAGUE                 PLAYER_ID POSITION  SEASON  AGE_SEPT_15  GP  G   A  \
0    WHL           100056/tye-hand        D    2013    18.252778  31  0   2   
1    OHL      100141/sean-thompson        F    2004    20.625000  38  6   5   
2   USHL  100151/stanislav-dzakhov        F    2013    20.497222  22  3   3   
3  QMJHL    100165/owen-bennington        F    2015    20.444444  66  2   2   
4  QMJHL      100224/tomas-pavelka        D    2013    20.294444  56  8  31   

   TP       PPG  ...  SEASON_>20GP  CUM_GP  ONLY_1_SEASON  MIN_3_SEASONS  \
0   2  0.060000  ...             1      31              1              0   
1  11  0.289474  ...             3     137              0              1   
2   6  0.272727  ...             2      66              0              0   
3   4  0.060000  ...             2     125              0              0   
4  39  0.696429  ...             2     123              0              0   

   DY_STATUS SHOOTS DRAFTED NHL_PPG  NHL_GP  NHL_DV  
0         DY      L      No     0.0       0       0  
1       DY+2      L      No     0.0       0       0  
2       DY+2      R      No     0.0       0       0  
3       DY+2      R      No     0.0       0       0  
4       DY+2      L      No     0.0       0       0  

[5 rows x 21 columns]

In [23]:
# Dropping columns that won't impact the prediction model(player ID, season) and those which will give it the answer(NHL games played and points)
df = df.drop(['PLAYER_ID', 'SEASON', 'NHL_PPG','NHL_GP','NHL_DV','LEAGUE','SHOOTS'], axis=1)
df.head()

POSITION  AGE_SEPT_15  GP  G   A  TP       PPG  SEASON_NO  SEASON_>20GP  \
0        D    18.252778  31  0   2   2  0.060000          1             1   
1        F    20.625000  38  6   5  11  0.289474          3             3   
2        F    20.497222  22  3   3   6  0.272727          2             2   
3        F    20.444444  66  2   2   4  0.060000          2             2   
4        D    20.294444  56  8  31  39  0.696429          2             2   

   CUM_GP  ONLY_1_SEASON  MIN_3_SEASONS DY_STATUS DRAFTED  
0      31              1              0        DY      No  
1     137              0              1      DY+2      No  
2      66              0              0      DY+2      No  
3     125              0              0      DY+2      No  
4     123              0              0      DY+2      No

In [24]:
# Convert Position to int (Forward = 0, Defense = 1, Goalie= 2)
df.POSITION[df.POSITION == 'F'] = 0
df.POSITION[df.POSITION == 'D'] = 1
df.POSITION[df.POSITION == 'G'] = 2


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [35]:
# Convert Draft year to int (draft year = 2, dy-2 = 0, etc)
df.DY_STATUS[df.DY_STATUS == 'DY-3'] = 0
df.DY_STATUS[df.DY_STATUS == 'DY-2'] = 1
df.DY_STATUS[df.DY_STATUS == 'DY-1'] = 2
df.DY_STATUS[df.DY_STATUS == 'DY'] = 3
df.DY_STATUS[df.DY_STATUS == 'DY+1'] = 4
df.DY_STATUS[df.DY_STATUS == 'DY+2'] = 5
df.DY_STATUS[df.DY_STATUS == 'DY+3'] = 6
df.DY_STATUS[df.DY_STATUS == 'DY+4'] = 7
df.head(10)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\lib\

POSITION  AGE_SEPT_15  GP   G   A  TP       PPG  SEASON_NO  SEASON_>20GP  \
0        1    18.252778  31   0   2   2  0.060000          1             1   
1        0    20.625000  38   6   5  11  0.289474          3             3   
2        0    20.497222  22   3   3   6  0.272727          2             2   
3        0    20.444444  66   2   2   4  0.060000          2             2   
4        1    20.294444  56   8  31  39  0.696429          2             2   
5        0    17.822222  21   0   1   1  0.050000          1             1   
6        0    21.605556  71  21  28  49  0.690141          4             4   
7        0    20.572222   8   1   0   1  0.130000          1             0   
8        1    20.777778  59   8  12  20  0.338983          2             1   
9        1    20.566667  51   3   9  12  0.235294          3             3   

   CUM_GP  ONLY_1_SEASON  MIN_3_SEASONS DY_STATUS DRAFTED  
0      31              1              0         3      No  
1     137              0              1         5      No  
2      66              0              0         5      No  
3     125              0              0         5      No  
4     123              0              0         5      No  
5      21              1              0         2      No  
6     284              0              1         6      No  
7       8              1              0         5      No  
8      78              0              0         5      No  
9     182              0              1         5      No

In [36]:
# split data into inputs and outputs
data = df.values
inputs = data[:, 0:13]
outputs = data[:, 13]

In [37]:
# split data into inputs and outputs
outcomes = df['DRAFTED']
outcomes.head()

input_factors = df.drop(['DRAFTED'], axis = 1)
input_factors.head()
feature_names = input_factors.columns

In [38]:
# Create Train/test groups
input_train, input_test, output_train, output_test = train_test_split(input_factors, outcomes, random_state=42)

In [39]:
# Test for Nan
pd.isnull(input_train).sum() > 0

POSITION         False
AGE_SEPT_15      False
GP               False
G                False
A                False
TP               False
PPG              False
SEASON_NO        False
SEASON_>20GP     False
CUM_GP           False
ONLY_1_SEASON    False
MIN_3_SEASONS    False
DY_STATUS        False
dtype: bool

In [40]:
pd.isnull(input_test).sum() > 0

POSITION         False
AGE_SEPT_15      False
GP               False
G                False
A                False
TP               False
PPG              False
SEASON_NO        False
SEASON_>20GP     False
CUM_GP           False
ONLY_1_SEASON    False
MIN_3_SEASONS    False
DY_STATUS        False
dtype: bool

In [41]:
# Fit and test decision tree model
clf = tree.DecisionTreeClassifier()
clf = clf.fit(input_train, output_train)
clf.score(input_test, output_test)

0.8005714285714286

In [42]:
# New Random Forest Decision tree
from sklearn.ensemble import RandomForestClassifier
randomforest = RandomForestClassifier(n_estimators=200)
randomforest = randomforest.fit(input_train, output_train)
randomforest.score(input_test, output_test)

0.8608571428571429

In [43]:
# Feature importance test
sorted(zip(randomforest.feature_importances_, feature_names), reverse=True)

[(0.1747921780488364, 'PPG'),
 (0.16687001637346305, 'AGE_SEPT_15'),
 (0.12185653629216159, 'A'),
 (0.11908830611330971, 'CUM_GP'),
 (0.1105045905038003, 'TP'),
 (0.09134809115798338, 'G'),
 (0.09074253938913497, 'GP'),
 (0.0443224515279253, 'DY_STATUS'),
 (0.023733191099653838, 'SEASON_>20GP'),
 (0.023039606838445362, 'POSITION'),
 (0.02173878479390729, 'SEASON_NO'),
 (0.007554168193408608, 'MIN_3_SEASONS'),
 (0.004409539667970264, 'ONLY_1_SEASON')]